In [ ]:
!pip install graphiti-core neo4j google-generativeai python-dotenv

In [ ]:
import os
import asyncio
from datetime import datetime, timezone
from dotenv import load_dotenv

import google.generativeai as genai
from neo4j import AsyncGraphDatabase
from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType

# Load environment variables
load_dotenv()

# Set up Gemini
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.5-flash")

# Neo4j setup
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")


class ChatMemory:
    def __init__(self, uri, user, password):
        self.driver = AsyncGraphDatabase.driver(uri, auth=(user, password))
        self.graphiti = Graphiti(uri=uri, user=user, password=password)

    async def save_message_as_episode(self, user_id: str, text: str, sender: str):
        timestamp = datetime.now(timezone.utc).isoformat()

        await self.graphiti.add_episode(
            external_user_id=user_id,
            content=text,
            sender=sender,
            type=EpisodeType.chat,
            timestamp=timestamp,
        )


async def run_conversation():
    memory = ChatMemory(neo4j_uri, neo4j_user, neo4j_password)

    user_id = "user-123"
    user_input = "I need help with my order"

    # Step 1: Save user message to memory
    await memory.save_message_as_episode(user_id, user_input, sender="user")

    # Step 2: Gemini reply
    response = gemini_model.generate_content(user_input)
    reply = response.text.strip()

    # Step 3: Save Gemini response to memory
    await memory.save_message_as_episode(user_id, reply, sender="agent")

    print("Gemini Reply:", reply)


# Run async main
asyncio.run(run_conversation())
